# Upload dos arquivos necessários ao google colab

* O primeiro arquivo é a pasta no formato .zip com as imagens
* O segundo arquivo é o modelo final gerado pelo treinamento (*trees-vs-soil-CNN.h5*)

Caminho da pasta: '/content/XMB/nome_da_pasta/'

Caminho do arquivo (já inserido no presente script): '/content/trees-vs-soil-CNN.h5'

In [2]:
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving Data.zip to Data.zip


Saving trees-vs-soil-CNN.h5 to trees-vs-soil-CNN.h5


In [3]:
import shutil
zipfile = '/content/Data.zip'
unzip_to = '/content/XMB/'
shutil.unpack_archive(zipfile, unzip_to)

# Script de inferência de Rede:

Apenas por demonstração, a pasta utilizada para inferência de rede foi a pasta de dados disponibilizada para o case.

In [4]:
# -*- coding: utf-8 -*-
"""
@author: Juan S Correa
"""
import os
import cv2
import csv
import numpy as np
from tensorflow.keras.models import load_model

def avg_on_missing_values(bad_img, print_media = False):
  R_sum = 0
  G_sum = 0
  B_sum = 0
  k = 0

  # obtendo a média de cada componente RGB dos pixels não falhos.
  for i in range(len(bad_img)):
    for j in range(len(bad_img[i])):
      if bad_img[i,j].any() != 0:
        k = k+1

        R_sum = R_sum + bad_img[i,j,0]
        G_sum = G_sum + bad_img[i,j,1]
        B_sum = B_sum + bad_img[i,j,2]

  R_mean = R_sum/k
  G_mean = G_sum/k
  B_mean = B_sum/k

  pixel_mean = np.array([R_mean,G_mean,B_mean])

  if print_media:
    print(f'[R={pixel_mean[0]:.1f},  G={pixel_mean[1]:.1f}, B={pixel_mean[2]:.1f} ]')

  # substituindo os pixels falhos pela média obtida.
  for i in range(len(bad_img)):
    for j in range(len(bad_img[i])):
      if bad_img[i,j].any() == 0:
        bad_img[i,j] = pixel_mean.astype('uint8')
  
  return(bad_img)

def normalize_rgb(rgb_img):  
  rgb_img = rgb_img.astype('float32')

  # recuperando uma lista separada para cada componente RGB do pixel
  r = rgb_img[:,:,0]
  g = rgb_img[:,:,1]
  b = rgb_img[:,:,2]

  norm_img = np.zeros(rgb_img.shape, np.float32)

  # realizando a operação de normalização para cada pixel
  for i in range(len(rgb_img)):
    for j in range(len(rgb_img[i])):

      sum = r[i,j] + g[i,j] + b[i,j]
      
      norm_img[i,j] = np.array([r[i,j]/sum, g[i,j]/sum, b[i,j]/sum])
        
  return norm_img

#-------------------------------------------------------------------------------

user_input = input('Caminho da pasta:')

model = load_model('/content/trees-vs-soil-CNN.h5')

DATADIR = user_input
CATEGORIAS = ['solo','arvore']

# lista com o nome das imagens
imgs_list = os.listdir(DATADIR)

data_list = []
for name in imgs_list:
  img_array = cv2.imread(os.path.join(DATADIR,name), cv2.IMREAD_COLOR)
  img_array_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

  data_list.append([name, img_array_rgb])

# processando imagens com pixels falhos
data_list_corrected = []
for name, img in data_list:
  data_list_corrected.append([name, avg_on_missing_values(img)])

# normalizando imagens
norm_data_list = []
for name, img in data_list_corrected:
  norm_data_list.append([name, normalize_rgb(img)])

# finalizando a preparação do input do modelo
X = []
IMG_SIZE = norm_data_list[0][1].shape[0]

for name, features in norm_data_list:
  X.append(features)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print("Imagens processadas com sucesso!")
print("Iniciando previsão...")

# realizando previsao
y_pred = model.predict(X)

y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

lista = []
for i in range(len(y_pred)):
  lista.append(CATEGORIAS[int(y_pred[i][0])])

output = []
for img, previsao in zip(imgs_list,lista):
  output.append([img, previsao])

print("Previsão realizada com sucesso!")

# exportando arquivo csv com previsão
with open('predictions.csv', mode='w') as output_file:
    writer = csv.writer(output_file)
    writer.writerows(output)

print("Arquivo salvo como: predictions.csv")

Caminho da pasta:/content/XMB/Data/
Imagens processadas com sucesso!
Iniciando previsão...
209/209 [==============================] - 6s 27ms/step
Previsão realizada com sucesso!
Arquivo salvo como: predictions.csv
